In [1]:
from pymongo.mongo_client import MongoClient
import os
from dotenv import load_dotenv

load_dotenv()

uri = os.getenv("mongodb_uri")
# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:

    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [2]:
# Choose 'appetit_db' database
db = client['appetit_db']

# List collections in 'appetit_db'
collections = db.list_collection_names()
print(f"Collections in appetit_db:", collections)


Collections in appetit_db: ['agenda_schedule', 'agenda_schedules', 'asap_schedules', 'categories', 'chowlydetails', 'customer_logs', 'delivery_fees', 'delivery_reservation_availabilities', 'delivery_settlements', 'dine_reservation_availabilities', 'locations', 'merchant_logs', 'merchant_settings', 'orders', 'pickup_reservation_availabilities', 'product_categories', 'product_common_modifiers', 'products', 'promo_codes', 'promos', 'refreshtokens', 'search_filters', 'settlements', 'shipday_logs', 'store_delivery_time_slots', 'store_dinein_time_slots', 'store_pickup_time_slots', 'store_schedules', 'store_tables', 'stores', 'tags', 'users', 'versions', 'voucher_codes', 'vouchers']


In [7]:
import datetime
from typing import Dict, List, Any
from decimal import Decimal, InvalidOperation

def safe_decimal_convert(value) -> Decimal:
    """
    Safely convert a value to Decimal, handling None and invalid values
    """
    if value is None:
        return Decimal('0')
    try:
        if isinstance(value, (int, float)):
            return Decimal(str(value))
        return Decimal(str(value))
    except (InvalidOperation, TypeError, ValueError):
        return Decimal('0')

def safe_division(numerator: Decimal, denominator: Decimal) -> Decimal:
    """
    Safely perform division, handling division by zero
    """
    try:
        if denominator == 0:
            return Decimal('0')
        return numerator / denominator
    except (InvalidOperation, TypeError, ValueError):
        return Decimal('0')

def get_least_performing_restaurants(db) -> Dict[str, Any]:
    """
    Fetches least performing restaurants based on order volume,
    including both volume and value metrics
    """
    pipeline = [
        {
            "$match": {
                "createdAt": {
                    "$gte": datetime.datetime.now() - datetime.timedelta(days=180),
                    "$lt": datetime.datetime.now()
                }
            }
        },
        {
            "$group": {
                "_id": "$store_id",
                "total_orders": {"$sum": 1},
                "total_value": {"$sum": "$orderValue"},
                "average_order_value": {"$avg": "$orderValue"},
                "monthly_stats": {
                    "$push": {
                        "month": {"$month": "$createdAt"},
                        "year": {"$year": "$createdAt"},
                        "value": "$orderValue"
                    }
                }
            }
        },
        {
            "$lookup": {
                "from": "stores",
                "localField": "_id",
                "foreignField": "_id",
                "as": "store_details"
            }
        },
        {
            "$unwind": "$store_details"
        },
        {
            "$project": {
                "store_name": "$store_details.name",
                "total_orders": 1,
                "total_value": 1,
                "average_order_value": 1,
                "monthly_stats": 1
            }
        },
        {
            "$sort": {"total_orders": 1}
        },
        {
            "$limit": 10
        }
    ]

    results = list(db.orders.aggregate(pipeline))
    
    def process_monthly_stats(stats):
        monthly_dict = {}
        for stat in stats:
            month_key = f"{stat.get('year', 0)}-{stat.get('month', 0):02d}"
            if month_key not in monthly_dict:
                monthly_dict[month_key] = {
                    "orders": 0,
                    "value": Decimal('0')
                }
            monthly_dict[month_key]["orders"] += 1
            monthly_dict[month_key]["value"] += safe_decimal_convert(stat.get('value', 0))
        
        return [
            {
                "month": k,
                "orders": v["orders"],
                "value": float(round(v["value"], 2))
            }
            for k, v in sorted(monthly_dict.items())
        ]

    # Calculate overall metrics with safe conversion
    total_system_volume = Decimal(sum(r.get("total_orders", 0) for r in results))
    total_system_value = sum(safe_decimal_convert(r.get("total_value", 0)) for r in results)
    
    return {
        "restaurant_breakdown": [
            {
                "store_name": r.get("store_name", "Unknown"),
                "metrics": {
                    "total_orders": r.get("total_orders", 0),
                    "total_value": float(round(safe_decimal_convert(r.get("total_value", 0)), 2)),
                    "average_order_value": float(round(safe_decimal_convert(r.get("average_order_value", 0)), 2)),
                    "volume_percentage": float(round(safe_division(
                        safe_decimal_convert(r.get("total_orders", 0)) * Decimal('100'),
                        total_system_volume
                    ), 2)),
                    "value_percentage": float(round(safe_division(
                        safe_decimal_convert(r.get("total_value", 0)) * Decimal('100'),
                        total_system_value
                    ), 2))
                },
                "monthly_performance": process_monthly_stats(r.get("monthly_stats", []))
            }
            for r in results
        ],
        "aggregated_metrics": {
            "total_volume": int(total_system_volume),
            "total_value": float(round(total_system_value, 2)),
            "average_order_value": float(round(
                safe_division(total_system_value, total_system_volume), 
                2
            )),
            "analysis_period": {
                "start_date": (datetime.datetime.now() - datetime.timedelta(days=180)).strftime("%Y-%m-%d"),
                "end_date": datetime.datetime.now().strftime("%Y-%m-%d")
            },
            "restaurants_analyzed": len(results)
        },
        "performance_indicators": {
            "volume_threshold": float(round(
                safe_division(total_system_volume, Decimal(str(len(results) if results else 1))),
                2
            )),
            "value_threshold": float(round(
                safe_division(total_system_value, Decimal(str(len(results) if results else 1))),
                2
            ))
        }
    }

In [8]:
get_least_performing_restaurants(db)

{'restaurant_breakdown': [{'store_name': "Hanzlian's Homemade Sausage",
   'metrics': {'total_orders': 2,
    'total_value': 0.0,
    'average_order_value': 0.0,
    'volume_percentage': 4.55,
    'value_percentage': 0.0},
   'monthly_performance': [{'month': '2024-10', 'orders': 1, 'value': 0.0},
    {'month': '2024-11', 'orders': 1, 'value': 0.0}]},
  {'store_name': 'Two Nines Kitchen & Bar',
   'metrics': {'total_orders': 2,
    'total_value': 0.0,
    'average_order_value': 0.0,
    'volume_percentage': 4.55,
    'value_percentage': 0.0},
   'monthly_performance': [{'month': '2024-09', 'orders': 2, 'value': 0.0}]},
  {'store_name': 'Oralia',
   'metrics': {'total_orders': 3,
    'total_value': 0.0,
    'average_order_value': 0.0,
    'volume_percentage': 6.82,
    'value_percentage': 0.0},
   'monthly_performance': [{'month': '2024-07', 'orders': 3, 'value': 0.0}]},
  {'store_name': 'La Verdad',
   'metrics': {'total_orders': 4,
    'total_value': 0.0,
    'average_order_value': 0.

In [13]:
def get_average_order_value_by_user(db) -> Dict[str, Any]:
    """
    Fetches average order value by user with additional metrics
    """
    pipeline = [
        {
            "$group": {
                "_id": "$user_id",
                "average_order_value": {"$avg": "$total_amount"},
                "total_orders": {"$sum": 1},
                "total_spent": {"$sum": "$total_amount"}
            }
        },
        {
            "$lookup": {
                "from": "users",
                "localField": "_id",
                "foreignField": "_id",
                "as": "user_details"
            }
        },
        {
            "$unwind": "$user_details"
        },
        {
            "$project": {
                "user_name": "$user_details.name",
                "average_order_value": 1,
                "total_orders": 1,
                "total_spent": 1
            }
        },
        {
            "$sort": {"average_order_value": -1}
        }
    ]

    results = list(db.orders.aggregate(pipeline))
    
    print(results[0])
    
    return {
        "user_metrics": [
            {
                "user_name": r["_id"],
                "average_order_value": float(round(Decimal(str(r["average_order_value"])), 2)),
                "total_orders": r["total_orders"],
                "total_spent": float(round(Decimal(str(r["total_spent"])), 2))
            }
            for r in results
        ],
        "aggregated_metrics": {
            "overall_average_order_value": float(round(Decimal(str(
                sum(r["total_spent"] for r in results) / sum(r["total_orders"] for r in results)
            )), 2)),
            "total_users": len(results),
            "total_orders": sum(r["total_orders"] for r in results)
        }
    }

In [14]:
get_average_order_value_by_user(db)

{'_id': ObjectId('659d7b886225e63b82132f0f'), 'average_order_value': 650.9, 'total_orders': 1, 'total_spent': 650.9}


InvalidOperation: [<class 'decimal.ConversionSyntax'>]

# Monthly orders for each restaurant in 2024

In [15]:

import datetime  # Add this import for datetime
import json


pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": datetime.datetime(2024, 1, 1), "$lt": datetime.datetime(2025, 1, 1)}
        }
    },
    {
        "$group": {
            "_id": {
                "month": {"$month": "$createdAt"},
                "store_id": "$store_id"
            },
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "stores",  # adjust if your collection name differs
            "localField": "_id.store_id",
            "foreignField": "_id",
            "as": "store_details"
        }
    },
    {
        "$unwind": "$store_details"
    },
    {
        "$project": {
            "month": "$_id.month",
            "store_name": "$store_details.name",
            "total_orders": 1
        }
    },
    {
        "$sort": {"month": 1, "store_name": 1}
    }
]




results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str)) 


[
  {
    "_id": {
      "month": 1,
      "store_id": "5ffe033b5465ffa6c93cde70"
    },
    "total_orders": 28,
    "month": 1,
    "store_name": "Aguacates Mexican Bar & Grill | Elmwood"
  },
  {
    "_id": {
      "month": 1,
      "store_id": "5fb5bf5934d2dc3679565cac"
    },
    "total_orders": 21,
    "month": 1,
    "store_name": "Ballyhoo"
  },
  {
    "_id": {
      "month": 1,
      "store_id": "64fb74ecd347663f1e74be2c"
    },
    "total_orders": 55,
    "month": 1,
    "store_name": "Buffalo Deep Dish"
  },
  {
    "_id": {
      "month": 1,
      "store_id": "5fb57c6e34d2dc77f8565a76"
    },
    "total_orders": 12,
    "month": 1,
    "store_name": "Butera's Craft Beer & Craft Pizza"
  },
  {
    "_id": {
      "month": 1,
      "store_id": "5fc5707c0642fe105f7bf56d"
    },
    "total_orders": 47,
    "month": 1,
    "store_name": "Cafe 59"
  },
  {
    "_id": {
      "month": 1,
      "store_id": "5f9c37a75c9bb54266ffafc1"
    },
    "total_orders": 4,
    "month": 1,
   

# least orders in past 6 months

In [18]:
#Current date and date 6 months ago
current_date = datetime.datetime.now()
six_months_ago = current_date - datetime.timedelta(days=180)

pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": six_months_ago}
        }
    },
    {
        "$group": {
            "_id": "$store_id",
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "stores",
            "localField": "_id",
            "foreignField": "_id",
            "as": "store_details"
        }
    },
    {
        "$unwind": "$store_details"
    },
    {
        "$sort": {"total_orders": 1}  # Sort by total orders ascending to find the biggest declines
    },
    {
        "$limit": 5  # Limit to top 5 stores with the biggest declines
    },
    {
        "$project": {
            "store_name": "$store_details.name",
            "total_orders": 1
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))

[
  {
    "_id": "63cfefbd415cbf4fcae7a83a",
    "total_orders": 2,
    "store_name": "Hanzlian's Homemade Sausage"
  },
  {
    "_id": "66c74fe7bea643fc4b12caf0",
    "total_orders": 2,
    "store_name": "Two Nines Kitchen & Bar"
  },
  {
    "_id": "671a52a2a81cf2b25e485b66",
    "total_orders": 2,
    "store_name": "Egyptian Bites"
  },
  {
    "_id": "66854adc230c2416d875e4f1",
    "total_orders": 3,
    "store_name": "Oralia"
  },
  {
    "_id": "6567471925523d16798c60ea",
    "total_orders": 4,
    "store_name": "La Verdad"
  }
]


# Highest orders in the past 6 months

In [19]:
# Current date and date 6 months ago
current_date = datetime.datetime.now()
six_months_ago = current_date - datetime.timedelta(days=180)

pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": six_months_ago}
        }
    },
    {
        "$group": {
            "_id": "$store_id",
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "stores",
            "localField": "_id",
            "foreignField": "_id",
            "as": "store_details"
        }
    },
    {
        "$unwind": "$store_details"
    },
    {
        "$sort": {"total_orders": -1}  # Sort by total orders descending to find the highest orders
    },
    {
        "$limit": 5  # Optionally limit to top 5 stores with the highest orders
    },
    {
        "$project": {
            "store_name": "$store_details.name",
            "total_orders": 1
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str)) 

[
  {
    "_id": "64fb74ecd347663f1e74be2c",
    "total_orders": 315,
    "store_name": "Buffalo Deep Dish"
  },
  {
    "_id": "5fc5707c0642fe105f7bf56d",
    "total_orders": 239,
    "store_name": "Cafe 59"
  },
  {
    "_id": "5fa18d605c9bb56497ffb1ee",
    "total_orders": 165,
    "store_name": "Fat Bob's Smokehouse"
  },
  {
    "_id": "5fda363171684a8075618c44",
    "total_orders": 158,
    "store_name": "Glen Park Tavern "
  },
  {
    "_id": "6422ecc5a7f10a43c4ff6baf",
    "total_orders": 141,
    "store_name": "Grass Fed Vegan Butcher Shop"
  }
]


# Highest number of orders by user zip code  for past 6 months

In [38]:


pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": datetime.datetime.now() - datetime.timedelta(days=180)},  # Last 6 months
            "zip_code": {"$exists": True, "$ne": ""}  # Ensures that the zip_code field exists and is not empty
        }
    },
    {
        "$group": {
            "_id": "$zip_code",
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$sort": {"total_orders": -1}  # Sort by the number of orders descending
    },
    {
        "$limit": 5  # Limits the output to the top 5 entries
    },
    {
        "$project": {
            "zip_code": "$_id",
            "total_orders": 1
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))  # Use default=str to handle datetime objects


[
  {
    "_id": null,
    "total_orders": 1184,
    "zip_code": null
  },
  {
    "_id": "14075",
    "total_orders": 156,
    "zip_code": "14075"
  },
  {
    "_id": "14213",
    "total_orders": 139,
    "zip_code": "14213"
  },
  {
    "_id": "14222",
    "total_orders": 112,
    "zip_code": "14222"
  },
  {
    "_id": "14216",
    "total_orders": 79,
    "zip_code": "14216"
  }
]


# least orders in past 6 months wrt to user zip codes

In [39]:
pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": datetime.datetime.now() - datetime.timedelta(days=180)},  # Last 6 months
            "zip_code": {"$exists": True, "$ne": ""}  # Ensures that the zip_code field exists and is not empty
        }
    },
    {
        "$group": {
            "_id": "$zip_code",
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$sort": {"total_orders": 1}  # Sort by the number of orders ascending
    },
    {
        "$limit": 5  # Limits the output to the least 5 entries
    },
    {
        "$project": {
            "zip_code": "$_id",
            "total_orders": 1
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))

[
  {
    "_id": "14425",
    "total_orders": 1,
    "zip_code": "14425"
  },
  {
    "_id": "14608",
    "total_orders": 1,
    "zip_code": "14608"
  },
  {
    "_id": "14612",
    "total_orders": 1,
    "zip_code": "14612"
  },
  {
    "_id": "14580",
    "total_orders": 1,
    "zip_code": "14580"
  },
  {
    "_id": "14068",
    "total_orders": 1,
    "zip_code": "14068"
  }
]


# orders acc to zip of  restaurants

In [40]:


pipeline = [
    {
        "$lookup": {
            "from": "stores",  # Assuming the name of the collection where store details are stored
            "localField": "store_id",
            "foreignField": "_id",
            "as": "store_info"
        }
    },
    {
        "$unwind": "$store_info"
    },
    {
        "$match": {
            "createdAt": {"$gte": datetime.datetime.now() - datetime.timedelta(days=180)},  # Last 6 months
            "store_info.zip_code": {"$exists": True, "$ne": ""}  # Ensures the zip_code field exists and is not empty
        }
    },
    {
        "$group": {
            "_id": "$store_info.zip_code",
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$sort": {"total_orders": -1}  # Sort by the number of orders descending
    },
    {
        "$project": {
            "zip_code": "$_id",
            "total_orders": 1
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))  # Use default=str to handle datetime objects


[
  {
    "_id": "14202",
    "total_orders": 556,
    "zip_code": "14202"
  },
  {
    "_id": "14204",
    "total_orders": 414,
    "zip_code": "14204"
  },
  {
    "_id": "14213",
    "total_orders": 339,
    "zip_code": "14213"
  },
  {
    "_id": "14127",
    "total_orders": 238,
    "zip_code": "14127"
  },
  {
    "_id": "14221",
    "total_orders": 207,
    "zip_code": "14221"
  },
  {
    "_id": "14222",
    "total_orders": 145,
    "zip_code": "14222"
  },
  {
    "_id": "14620",
    "total_orders": 141,
    "zip_code": "14620"
  },
  {
    "_id": "14075",
    "total_orders": 140,
    "zip_code": "14075"
  },
  {
    "_id": "14219",
    "total_orders": 137,
    "zip_code": "14219"
  },
  {
    "_id": "14607",
    "total_orders": 125,
    "zip_code": "14607"
  },
  {
    "_id": "14224",
    "total_orders": 105,
    "zip_code": "14224"
  },
  {
    "_id": "14052",
    "total_orders": 55,
    "zip_code": "14052"
  },
  {
    "_id": "14226",
    "total_orders": 48,
    "zip_code":

# New users in last 3 months

In [41]:


three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": three_months_ago}
        }
    },
    {
        "$group": {
            "_id": None,  # Grouping by None means counting all documents that match the previous stage
            "new_users_count": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "new_users_count": 1
        }
    }
]

results = db.users.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))  # Use default=str to handle datetime objects

[
  {
    "new_users_count": 737
  }
]


In [42]:
one_year_ago = datetime.datetime.now() - datetime.timedelta(days=365)

pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": one_year_ago}
        }
    },
    {
        "$group": {
            "_id": {
                "year": {"$year": "$createdAt"},
                "month": {"$month": "$createdAt"}
            },
            "new_users_count": {"$sum": 1}
        }
    },
    {
        "$sort": {"_id.year": 1, "_id.month": 1}  # Sort by year and month
    },
    {
        "$project": {
            "year": "$_id.year",
            "month": "$_id.month",
            "new_users_count": 1,
            "_id": 0
        }
    }
]

results = db.users.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str)) 

[
  {
    "new_users_count": 216,
    "year": 2023,
    "month": 11
  },
  {
    "new_users_count": 303,
    "year": 2023,
    "month": 12
  },
  {
    "new_users_count": 283,
    "year": 2024,
    "month": 1
  },
  {
    "new_users_count": 326,
    "year": 2024,
    "month": 2
  },
  {
    "new_users_count": 299,
    "year": 2024,
    "month": 3
  },
  {
    "new_users_count": 214,
    "year": 2024,
    "month": 4
  },
  {
    "new_users_count": 250,
    "year": 2024,
    "month": 5
  },
  {
    "new_users_count": 253,
    "year": 2024,
    "month": 6
  },
  {
    "new_users_count": 210,
    "year": 2024,
    "month": 7
  },
  {
    "new_users_count": 239,
    "year": 2024,
    "month": 8
  },
  {
    "new_users_count": 234,
    "year": 2024,
    "month": 9
  },
  {
    "new_users_count": 261,
    "year": 2024,
    "month": 10
  },
  {
    "new_users_count": 63,
    "year": 2024,
    "month": 11
  }
]


# Average Order Value by User

In [46]:
pipeline = [
    {
        "$group": {
            "_id": "$user_id",
            "average_order_value": {"$avg": "$total_amount"}
        }
    },
    {
        "$sort": {"average_order_value": -1}
    }
]

results = db.orders.aggregate(pipeline)
for result in results:
    print(result)

{'_id': ObjectId('659d7b886225e63b82132f0f'), 'average_order_value': 650.9}
{'_id': ObjectId('65b8044b479e30c46d21dc3f'), 'average_order_value': 631.79}
{'_id': ObjectId('65e9b039382f3846dfba9e1f'), 'average_order_value': 539.99}
{'_id': ObjectId('66648b91230c24639d5f9a3e'), 'average_order_value': 517.21}
{'_id': ObjectId('64b0051ae2bd6e6d013a3ea6'), 'average_order_value': 504.83}
{'_id': ObjectId('655cd048ebde7dccfff93771'), 'average_order_value': 487.58500000000004}
{'_id': ObjectId('62dff749a7d7d4899122d75f'), 'average_order_value': 484.365}
{'_id': ObjectId('62e15518a7d7d4364923ac86'), 'average_order_value': 481.43}
{'_id': ObjectId('66f44ec0819cc870c81b1245'), 'average_order_value': 448.2}
{'_id': ObjectId('64fb49c3d347664cf3748aaa'), 'average_order_value': 418.39}
{'_id': ObjectId('64524078d1bf7304bb6ec4ca'), 'average_order_value': 406.2875}
{'_id': ObjectId('621e342a0d4469af2f19af70'), 'average_order_value': 396.22}
{'_id': ObjectId('6601937dc2f3344339acbee8'), 'average_order_va

# year over growth restaurant wise

In [17]:
# Not sure about this
pipeline = [
    {
        "$match": {
            "createdAt": {"$gte": datetime.datetime(2020, 1, 1)}  # Starting from 2020 
        }
    },
    {
        "$group": {
            "_id": {
                "year": {"$year": "$createdAt"},
                "store_id": "$store_id"
            },
            "total_orders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "stores",
            "localField": "_id.store_id",
            "foreignField": "_id",
            "as": "store_details"
        }
    },
    {
        "$unwind": "$store_details"
    },
    {
        "$sort": {"_id.year": 1, "store_details.name": 1}
    },
    {
        "$group": {
            "_id": "$store_details.name",
            "orders_per_year": {
                "$push": {
                    "year": "$_id.year",
                    "orders": "$total_orders"
                }
            }
        }
    },
    {
        "$project": {
            "store_name": "$_id",
            "year_over_year_growth": {
                "$map": {
                    "input": {"$range": [0, {"$subtract": [{"$size": "$orders_per_year"}, 1]}]},
                    "as": "idx",
                    "in": {
                        "year": {"$arrayElemAt": ["$orders_per_year.year", {"$add": ["$$idx", 1]}]},
                        "growth": {
                            "$subtract": [
                                {"$divide": [
                                    {"$subtract": [
                                        {"$arrayElemAt": ["$orders_per_year.orders", {"$add": ["$$idx", 1]}]},
                                        {"$arrayElemAt": ["$orders_per_year.orders", "$$idx"]}
                                    ]},
                                    {"$arrayElemAt": ["$orders_per_year.orders", "$$idx"]}
                                ]},
                                1
                            ]
                        }
                    }
                }
            }
        }
    }
]

results = db.orders.aggregate(pipeline)
results_list = list(results)  # Convert cursor to list

# Print results as JSON
print(json.dumps(results_list, indent=2, default=str))  # Use default=str to handle datetime objects


[
  {
    "_id": "Hombre Y Lobo",
    "store_name": "Hombre Y Lobo",
    "year_over_year_growth": [
      {
        "year": 2022,
        "growth": 4.333333333333333
      },
      {
        "year": 2023,
        "growth": -1.5
      },
      {
        "year": 2024,
        "growth": -1.526315789473684
      }
    ]
  },
  {
    "_id": "La Verdad",
    "store_name": "La Verdad",
    "year_over_year_growth": [
      {
        "year": 2024,
        "growth": 10.0
      }
    ]
  },
  {
    "_id": "Phyu Thein",
    "store_name": "Phyu Thein",
    "year_over_year_growth": []
  },
  {
    "_id": "Malkia and Co.",
    "store_name": "Malkia and Co.",
    "year_over_year_growth": []
  },
  {
    "_id": "Cafe 59",
    "store_name": "Cafe 59",
    "year_over_year_growth": [
      {
        "year": 2021,
        "growth": 9.48
      },
      {
        "year": 2022,
        "growth": -1.0116144018583042
      },
      {
        "year": 2023,
        "growth": -1.2679200940070505
      },
      {
 